## Importing Libraries

In [1]:
import tkinter as tk
from tkinter.filedialog import askopenfilename
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
import scipy.stats as stats
import numpy as np
from pandas.api.types import is_numeric_dtype
import statsmodels.api as sm 
from scipy.stats import gamma
from sklearn.preprocessing import StandardScaler
from keras.utils.np_utils import to_categorical
import pickle 
from sklearn.metrics import mean_squared_error
from sklearn.decomposition import PCA
#-------Importing tensorflow libraries-------#
from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense,Dropout
from tensorflow.python.keras.wrappers.scikit_learn import KerasRegressor
#------split and pipeline libraries------#
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import KFold
from sklearn.pipeline import Pipeline

Using TensorFlow backend.


## Feature Engineering and Model Training functions

In [2]:
def categorise_signal(row):
    """Feature Engineering to categorise signal strength for classifier model"""
    if row >= 7:
        return 3
    elif row <=4:
        return 1
    else:
        return 2
        
def classifier_model_architecture(input_dim,output_dim):
    """Defines the generic architecture of the classifier neural network"""
    model = Sequential()
    model.add(Dense(32, input_dim=input_dim, kernel_initializer='normal', activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(64, activation='relu'))
    model.add(Dropout(0.5))
    model.add(Dense(output_dim, activation='softmax'))
    return model

def principal_components(data):
    """Convert the input features into principal components"""
    scaled = data.copy()
    #----Column standardization---------#
    for column in scaled.drop(columns=['Signal_Strength']).columns:
        scaled[column] = scaled[column].apply(lambda x:
                        (x - scaled[column].mean())/scaled[column].std()
                       )
    #-----Decomposition into principal components at n_components = 6 which we found during EDA-----#
    features = list(scaled.drop(columns=['Signal_Strength']).columns)
    pca = PCA(n_components=6)
    principalComponents = pca.fit_transform(np.array(scaled[features]))
    pcDf = pd.DataFrame(data = principalComponents
             , columns = ['PC1', 'PC2','PC3','PC4','PC5','PC6'])
    finalDf = pd.concat([pcDf,scaled[['Signal_Strength']]], axis=1)
    return finalDf

def backwardElimination(x, Y, significance_level, columns):
    """Backward elimination method"""
    numVars = len(x[0])
    for i in range(0, numVars):
        regressor_OLS = sm.OLS(Y, x).fit()
        maxVar = max(regressor_OLS.pvalues)
        if maxVar > significance_level:
            for j in range(0, numVars - i):
                if (regressor_OLS.pvalues[j] == maxVar):
                    x = np.delete(x, j, 1)
                    columns = np.delete(columns, j)
                    
    regressor_OLS.summary()
    return x, columns

def import_csv_data():
    """Code to import csv data"""
    global v,csv_check,df
    csv_file_path = askopenfilename()
    print(csv_file_path)
    v.set(csv_file_path)
    try:
        csv_check.set("")
        df = pd.read_csv(csv_file_path)
        csv_check.set("Done")
    except:
        csv_check.set("Not a CSV")
    
def check_target():
    """To check if dependent variable is present or not"""
    global target,target_found
    if target.get() in df.columns:
        target_found.set("Found")
    else:
        target_found.set("Not Found")

def regressor_train():
    """Train Regressor model"""
    global model1,trained
    df1 = df.copy()
    #-----Imputing outliers------------#
    for column in df1.drop(columns=['Signal_Strength']).columns:
        df1[column] = df1[column].apply(lambda x: x if 
                        ((x - df1[column].mean())/df1[column].std()) <= 3 
                        else
                        df1[column].median()
                        #np.nan
                       )
    #------------Applying backward elimination to filter out columns----------#
    x_ols = df1.drop(columns=['Signal_Strength']).values
    y_ols = df1['Signal_Strength']
    significance_level = 0.05
    selected_columns = df1.drop(columns=['Signal_Strength']).columns
    data, selected_columns = backwardElimination(x_ols, y_ols, significance_level, selected_columns)
    #---------Retaining only selected columns obtained after backward elimination---------------------------#
    selected_columns = list(selected_columns)
    selected_columns.append('Signal_Strength')
    After_backward_elimination = df[selected_columns].copy()
    #---------------Column Standardization----------------------#
    for column in After_backward_elimination.drop(columns=['Signal_Strength']).columns:
        After_backward_elimination[column] = After_backward_elimination[column].apply(lambda x:
                        (x - After_backward_elimination[column].mean())/After_backward_elimination[column].std()
                       )
    #---------------------Data split--------------------------#
    X = After_backward_elimination.drop(columns=['Signal_Strength']).values
    Y = After_backward_elimination['Signal_Strength'].values
    X_train, X_test, y_train, y_test = train_test_split(X, Y,test_size=0.33, random_state=42)
    #--------Defining the Model architecture------------------------#
    model1 = Sequential()
    model1.add(Dense(10, input_dim=7, kernel_initializer='normal', activation='relu'))
    model1.add(Dropout(0.25))
    model1.add(Dense(10, activation='relu'))
    model1.add(Dropout(0.25))
    model1.add(Dense(1, activation='linear'))
    #-------------Training the model-------------#
    try:
        trained.set("")
        model1.compile(loss='mse', optimizer='adam', metrics=['mse','mae'])
        model1.fit(X_train, y_train, epochs=200, batch_size=10,  verbose=1, validation_split=0.2)
        trained.set("Network trained")
    except:
        trained.set("Error while Training the network")

def save_regressor_model():
    """Save Regressor model"""
    global saved
    try:
        saved.set("")
        model1.save("Signal_Regressor_model.h5")
        saved.set("Saved to the Disk")
    except:
        saved.set("Not able to save due to the error")

def classifier_train():
    """Train classifier model"""
    global model2, c_trained
    df2 = df.copy()
    #------Imputing outliers with median values-----#
    for column in df2.drop(columns=['Signal_Strength']).columns:
        df2[column] = df2[column].apply(lambda x: x if
                        ((x - df2[column].mean())/df2[column].std()) <= 3 
                        else
                        df2[column].median()
                       )
    #---------Feature engineering to categorise signal strength into 1,2 and 3-----------#
    df2['Signal_Strength'] = df2['Signal_Strength'].apply(categorise_signal)
    #----------getting principal components--------#
    finalDf = principal_components(df2)
    #------categorising the target variables and splitting the data------#
    categorical_labels = to_categorical(finalDf['Signal_Strength'], num_classes=None)
    xpca = finalDf.drop(columns=['Signal_Strength']).values
    ypca = categorical_labels
    X_train, X_test, y_train, y_test = train_test_split(xpca, ypca,test_size=0.33, random_state=42)
    #---------defining model architecture----------#
    model2 = classifier_model_architecture(6,4)
    #---------Train the model------------------#
    try:
        c_trained.set("")
        model2.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
        model2.fit(X_train, y_train, epochs=150, batch_size=10,  verbose=1, validation_split=0.2)
        c_trained.set("Network trained")
    except:
        c_trained.set("Error while Training the network")

def save_classifier_model():
    """Saving Classifier model"""
    global c_saved
    try:
        c_saved.set("")
        model2.save("Signal_Classification_model.h5")
        c_saved.set("Saved to the Disk")
    except:
        c_saved.set("Not able to save due to the error")

## Tkinter GUI code

In [3]:
#---------------This section defines our UI----------------------#
#----------------Step 1:---------------------------------------#
root = tk.Tk()
root.title('Neural Networks GUI - Great Learning')
tk.Label(root, text='Step 1: File Name').grid(row=5, column=0)
v = tk.StringVar()
csv_check = tk.StringVar()
entry = tk.Entry(root, textvariable=v).grid(row=5, column=1)
tk.Button(root, text='Import Data',command=import_csv_data).grid(row=5, column=2)
entry2 = tk.Entry(root, textvariable=csv_check).grid(row=5, column=3)
#tk.Button(root, text='Close',command=root.destroy).grid(row=10, column=5)
#----------------Step 2:-----------------#
tk.Label(root, text='Step 2: Target Column').grid(row=7, column=0)
target = tk.StringVar()
target_found = tk.StringVar()
userentry = tk.Entry(root, textvariable=target).grid(row=7, column=1)
tk.Button(root, text='Import Target',command=check_target).grid(row=7, column=2)
found_or_not = tk.Entry(root, textvariable=target_found).grid(row=7, column=3)
#----------------Step 3:-----------------#
#---------------------Regressor GUI--------------------------#
tk.Label(root, text='Step 3: Neural Network Regressor').grid(row=8, column=0)
tk.Label(root, text='Regression').grid(row=9, column=0)
tk.Label(root, text='Pickle').grid(row=10, column=0)
trained = tk.StringVar()
saved = tk.StringVar()
tk.Button(root, text='Train',command=regressor_train).grid(row=9, column=1)
tr_entry = tk.Entry(root, textvariable=trained).grid(row=9, column=2)
tk.Button(root, text='Run',command=save_regressor_model).grid(row=10, column=1)
savedentry = tk.Entry(root, textvariable=saved).grid(row=10, column=2)
#-------------Step 4------------------------------#
#-----------------Classifier GUI-----------------------#
tk.Label(root, text='Step 3: Neural Network Classifier').grid(row=11, column=0)
tk.Label(root, text='Classification').grid(row=12, column=0)
tk.Label(root, text='Pickle').grid(row=13, column=0)
c_trained = tk.StringVar()
c_saved = tk.StringVar()
tk.Button(root, text='Train',command=classifier_train).grid(row=12, column=1)
tr_entry_c = tk.Entry(root, textvariable=c_trained).grid(row=12, column=2)
tk.Button(root, text='Run',command=save_classifier_model).grid(row=13, column=1)
saved_c = tk.Entry(root, textvariable=c_saved).grid(row=13, column=2)
root.mainloop()

C:/Users/DELL/Downloads/GL_Assignment_NeuralNetwork/Part- 1,2&3 - Signal.csv
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 856 samples, validate on 215 samples
Epoch 1/200
856/856 [==============================] - 0s 391us/sample - loss: 28.0968 - mean_squared_error: 28.0967 - mean_absolute_error: 5.2218 - val_loss: 23.8504 - val_mean_squared_error: 23.8504 - val_mean_absolute_error: 4.7994
Epoch 2/200
856/856 [==============================] - 0s 119us/sample - loss: 16.8039 - mean_squared_error: 16.8039 - mean_absolute_error: 3.8827 - val_loss: 9.5178 - val_mean_squared_error: 9.5178 - val_mean_absolute_error: 2.8908
Epoch 3/200
856/856 [==============================] - 0s 103us/sample - loss: 5.9206 - mean_squared_error: 5.9206 - mean_absolute_error: 2.0649 - val_loss: 2.1910 - val_mean_squared_error: 2.1910 - val_mean_ab

Epoch 33/200
856/856 [==============================] - 0s 105us/sample - loss: 1.4783 - mean_squared_error: 1.4783 - mean_absolute_error: 0.9534 - val_loss: 0.3885 - val_mean_squared_error: 0.3885 - val_mean_absolute_error: 0.4712
Epoch 34/200
856/856 [==============================] - 0s 102us/sample - loss: 1.5172 - mean_squared_error: 1.5172 - mean_absolute_error: 0.9754 - val_loss: 0.3558 - val_mean_squared_error: 0.3558 - val_mean_absolute_error: 0.4483
Epoch 35/200
856/856 [==============================] - 0s 102us/sample - loss: 1.5423 - mean_squared_error: 1.5423 - mean_absolute_error: 0.9771 - val_loss: 0.4612 - val_mean_squared_error: 0.4612 - val_mean_absolute_error: 0.5194
Epoch 36/200
856/856 [==============================] - 0s 103us/sample - loss: 1.3351 - mean_squared_error: 1.3351 - mean_absolute_error: 0.9081 - val_loss: 0.3669 - val_mean_squared_error: 0.3669 - val_mean_absolute_error: 0.4606
Epoch 37/200
856/856 [==============================] - 0s 99us/sample -

856/856 [==============================] - 0s 106us/sample - loss: 1.1111 - mean_squared_error: 1.1111 - mean_absolute_error: 0.8298 - val_loss: 0.4097 - val_mean_squared_error: 0.4097 - val_mean_absolute_error: 0.4780
Epoch 69/200
856/856 [==============================] - 0s 102us/sample - loss: 0.9720 - mean_squared_error: 0.9720 - mean_absolute_error: 0.7741 - val_loss: 0.3557 - val_mean_squared_error: 0.3557 - val_mean_absolute_error: 0.4596
Epoch 70/200
856/856 [==============================] - 0s 102us/sample - loss: 1.0180 - mean_squared_error: 1.0180 - mean_absolute_error: 0.7963 - val_loss: 0.3817 - val_mean_squared_error: 0.3817 - val_mean_absolute_error: 0.4693
Epoch 71/200
856/856 [==============================] - 0s 104us/sample - loss: 0.9335 - mean_squared_error: 0.9335 - mean_absolute_error: 0.7580 - val_loss: 0.3537 - val_mean_squared_error: 0.3537 - val_mean_absolute_error: 0.4538
Epoch 72/200
856/856 [==============================] - 0s 102us/sample - loss: 1.013

856/856 [==============================] - 0s 103us/sample - loss: 0.7507 - mean_squared_error: 0.7507 - mean_absolute_error: 0.6734 - val_loss: 0.3521 - val_mean_squared_error: 0.3521 - val_mean_absolute_error: 0.4580
Epoch 104/200
856/856 [==============================] - 0s 114us/sample - loss: 0.7875 - mean_squared_error: 0.7875 - mean_absolute_error: 0.6932 - val_loss: 0.3737 - val_mean_squared_error: 0.3737 - val_mean_absolute_error: 0.4725
Epoch 105/200
856/856 [==============================] - 0s 99us/sample - loss: 0.8097 - mean_squared_error: 0.8097 - mean_absolute_error: 0.6896 - val_loss: 0.3629 - val_mean_squared_error: 0.3629 - val_mean_absolute_error: 0.4652
Epoch 106/200
856/856 [==============================] - 0s 100us/sample - loss: 0.7340 - mean_squared_error: 0.7340 - mean_absolute_error: 0.6666 - val_loss: 0.3669 - val_mean_squared_error: 0.3669 - val_mean_absolute_error: 0.4723
Epoch 107/200
856/856 [==============================] - 0s 102us/sample - loss: 0.

856/856 [==============================] - 0s 105us/sample - loss: 0.6507 - mean_squared_error: 0.6507 - mean_absolute_error: 0.6264 - val_loss: 0.3721 - val_mean_squared_error: 0.3721 - val_mean_absolute_error: 0.4720
Epoch 139/200
856/856 [==============================] - 0s 127us/sample - loss: 0.6376 - mean_squared_error: 0.6376 - mean_absolute_error: 0.6171 - val_loss: 0.3581 - val_mean_squared_error: 0.3581 - val_mean_absolute_error: 0.4632
Epoch 140/200
856/856 [==============================] - 0s 129us/sample - loss: 0.5711 - mean_squared_error: 0.5711 - mean_absolute_error: 0.5867 - val_loss: 0.3503 - val_mean_squared_error: 0.3503 - val_mean_absolute_error: 0.4567
Epoch 141/200
856/856 [==============================] - 0s 97us/sample - loss: 0.5951 - mean_squared_error: 0.5951 - mean_absolute_error: 0.5972 - val_loss: 0.3591 - val_mean_squared_error: 0.3591 - val_mean_absolute_error: 0.4641
Epoch 142/200
856/856 [==============================] - 0s 99us/sample - loss: 0.5

856/856 [==============================] - 0s 104us/sample - loss: 0.5192 - mean_squared_error: 0.5192 - mean_absolute_error: 0.5539 - val_loss: 0.3502 - val_mean_squared_error: 0.3502 - val_mean_absolute_error: 0.4605
Epoch 174/200
856/856 [==============================] - 0s 104us/sample - loss: 0.5241 - mean_squared_error: 0.5241 - mean_absolute_error: 0.5595 - val_loss: 0.3574 - val_mean_squared_error: 0.3574 - val_mean_absolute_error: 0.4653
Epoch 175/200
856/856 [==============================] - 0s 109us/sample - loss: 0.5277 - mean_squared_error: 0.5277 - mean_absolute_error: 0.5591 - val_loss: 0.3495 - val_mean_squared_error: 0.3495 - val_mean_absolute_error: 0.4626
Epoch 176/200
856/856 [==============================] - 0s 105us/sample - loss: 0.4955 - mean_squared_error: 0.4955 - mean_absolute_error: 0.5476 - val_loss: 0.3533 - val_mean_squared_error: 0.3533 - val_mean_absolute_error: 0.4624
Epoch 177/200
856/856 [==============================] - 0s 100us/sample - loss: 0

Epoch 13/150
856/856 [==============================] - 0s 104us/sample - loss: 0.4558 - acc: 0.8493 - val_loss: 0.4115 - val_acc: 0.8465
Epoch 14/150
856/856 [==============================] - 0s 104us/sample - loss: 0.4648 - acc: 0.8341 - val_loss: 0.4071 - val_acc: 0.8372
Epoch 15/150
856/856 [==============================] - 0s 104us/sample - loss: 0.4632 - acc: 0.8376 - val_loss: 0.4039 - val_acc: 0.8558
Epoch 16/150
856/856 [==============================] - 0s 102us/sample - loss: 0.4582 - acc: 0.8446 - val_loss: 0.4094 - val_acc: 0.8465
Epoch 17/150
856/856 [==============================] - 0s 105us/sample - loss: 0.4589 - acc: 0.8411 - val_loss: 0.4053 - val_acc: 0.8465
Epoch 18/150
856/856 [==============================] - 0s 103us/sample - loss: 0.4647 - acc: 0.8400 - val_loss: 0.4047 - val_acc: 0.8512
Epoch 19/150
856/856 [==============================] - 0s 106us/sample - loss: 0.4489 - acc: 0.8470 - val_loss: 0.4071 - val_acc: 0.8372
Epoch 20/150
856/856 [============

856/856 [==============================] - 0s 112us/sample - loss: 0.3979 - acc: 0.8540 - val_loss: 0.3927 - val_acc: 0.8512
Epoch 73/150
856/856 [==============================] - 0s 110us/sample - loss: 0.4241 - acc: 0.8645 - val_loss: 0.3954 - val_acc: 0.8512
Epoch 74/150
856/856 [==============================] - 0s 107us/sample - loss: 0.4113 - acc: 0.8505 - val_loss: 0.3905 - val_acc: 0.8465
Epoch 75/150
856/856 [==============================] - 0s 107us/sample - loss: 0.3959 - acc: 0.8540 - val_loss: 0.3902 - val_acc: 0.8512
Epoch 76/150
856/856 [==============================] - 0s 105us/sample - loss: 0.4194 - acc: 0.8540 - val_loss: 0.3870 - val_acc: 0.8558
Epoch 77/150
856/856 [==============================] - 0s 106us/sample - loss: 0.4079 - acc: 0.8493 - val_loss: 0.3901 - val_acc: 0.8558
Epoch 78/150
856/856 [==============================] - 0s 107us/sample - loss: 0.4037 - acc: 0.8493 - val_loss: 0.3883 - val_acc: 0.8465
Epoch 79/150
856/856 [=========================

856/856 [==============================] - 0s 116us/sample - loss: 0.3904 - acc: 0.8586 - val_loss: 0.3881 - val_acc: 0.8512
Epoch 132/150
856/856 [==============================] - 0s 111us/sample - loss: 0.3851 - acc: 0.8516 - val_loss: 0.3903 - val_acc: 0.8558
Epoch 133/150
856/856 [==============================] - 0s 110us/sample - loss: 0.3846 - acc: 0.8551 - val_loss: 0.3880 - val_acc: 0.8465
Epoch 134/150
856/856 [==============================] - 0s 105us/sample - loss: 0.3969 - acc: 0.8516 - val_loss: 0.3902 - val_acc: 0.8465
Epoch 135/150
856/856 [==============================] - 0s 106us/sample - loss: 0.3906 - acc: 0.8435 - val_loss: 0.3876 - val_acc: 0.8512
Epoch 136/150
856/856 [==============================] - 0s 116us/sample - loss: 0.4013 - acc: 0.8505 - val_loss: 0.3859 - val_acc: 0.8558
Epoch 137/150
856/856 [==============================] - 0s 109us/sample - loss: 0.3811 - acc: 0.8598 - val_loss: 0.3900 - val_acc: 0.8512
Epoch 138/150
856/856 [==================